In [6]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import os
import glob

# 读入数据

In [56]:
def readpoint(seq, i):
    path = '../datasets/dataset/sequences'
    path_seq = os.path.join(path, seq)
    label_path = os.path.join(path_seq, 'labels')
    point_path = os.path.join(path_seq, 'velodyne')
    labelset = sorted(glob.glob(os.path.join(label_path, '*.label')))
    dataset = sorted(glob.glob(os.path.join(point_path, '*.bin')))
    point_A = np.fromfile(dataset[i], dtype=np.float32)
    point_B = np.fromfile(dataset[i+1], dtype=np.float32)
    label_A = np.fromfile(labelset[i], dtype=np.float32)
    label_B = np.fromfile(labelset[i+1], dtype=np.float32)
    point_A = point_A.reshape((-1, 4))
    point_B = point_B.reshape((-1, 4))
    point_A = point_A[:, :3]
    point_B = point_B[:, :3]
    # for i in range(len(label_A)):
    #     if point_A[i, 2] < -1.6:
    #         label_A[i] = 0
    # for i in range(len(label_B)):
    #     if point_B[i, 2] < -1.6:
    #         label_B[i] = 0
    return point_A, point_B, label_A, label_B

# 1.标准ICP配准

In [53]:
import numpy as np
from sklearn.neighbors import NearestNeighbors


def best_fit_transform(A, B):
    '''
    Calculates the least-squares best-fit transform that maps corresponding points A to B in m spatial dimensions
    Input:
      A: Nxm numpy array of corresponding points
      B: Nxm numpy array of corresponding points
    Returns:
      T: (m+1)x(m+1) homogeneous transformation matrix that maps A on to B
      R: mxm rotation matrix
      t: mx1 translation vector
    '''

    # assert A.shape == B.shape

    # get number of dimensions
    m = A.shape[1]

    # translate points to their centroids
    centroid_A = np.mean(A, axis=0)
    centroid_B = np.mean(B, axis=0)
    AA = A - centroid_A
    BB = B - centroid_B

    # rotation matrix
    H = np.dot(AA.T, BB)
    U, S, Vt = np.linalg.svd(H)
    R = np.dot(Vt.T, U.T)

    # special reflection case
    if np.linalg.det(R) < 0:
       Vt[m-1,:] *= -1
       R = np.dot(Vt.T, U.T)

    # translation
    t = centroid_B.T - np.dot(R,centroid_A.T)

    # homogeneous transformation
    T = np.identity(m+1)
    T[:m, :m] = R
    T[:m, m] = t

    return T, R, t


def nearest_neighbor(src, dst):
    '''
    Find the nearest (Euclidean) neighbor in dst for each point in src
    Input:
        src: Nxm array of points
        dst: Nxm array of points
    Output:
        distances: Euclidean distances of the nearest neighbor
        indices: dst indices of the nearest neighbor
    '''

    # assert src.shape == dst.shape

    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(dst)
    distances, indices = neigh.kneighbors(src, return_distance=True)
    return distances.ravel(), indices.ravel()


def icp(A, B, init_pose=None, max_iterations=20, tolerance=0.001):
    '''
    The Iterative Closest Point method: finds best-fit transform that maps points A on to points B
    Input:
        A: Nxm numpy array of source mD points
        B: Nxm numpy array of destination mD point
        init_pose: (m+1)x(m+1) homogeneous transformation
        max_iterations: exit algorithm after max_iterations
        tolerance: convergence criteria
    Output:
        T: final homogeneous transformation that maps A on to B
        distances: Euclidean distances (errors) of the nearest neighbor
        i: number of iterations to converge
    '''

    # assert A.shape == B.shape

    # get number of dimensions
    m = A.shape[1]

    # make points homogeneous, copy them to maintain the originals
    src = np.ones((m+1,A.shape[0]))
    dst = np.ones((m+1,B.shape[0]))
    src[:m,:] = np.copy(A.T)
    dst[:m,:] = np.copy(B.T)

    # apply the initial pose estimation
    if init_pose is not None:
        src = np.dot(init_pose, src)

    prev_error = 0

    for i in range(max_iterations):
        # find the nearest neighbors between the current source and destination points
        distances, indices = nearest_neighbor(src[:m,:].T, dst[:m,:].T)

        # compute the transformation between the current source and nearest destination points
        T,_,_ = best_fit_transform(src[:m,:].T, dst[:m,indices].T)

        # update the current source
        src = np.dot(T, src)

        # check error
        mean_error = np.mean(distances)
        if np.abs(prev_error - mean_error) < tolerance:
            break
        prev_error = mean_error

    # calculate final transformation
    T,_,_ = best_fit_transform(A, src[:m,:].T)

    return T, distances, i

# 2.带权ICP配准

In [54]:
import numpy as np
from sklearn.neighbors import NearestNeighbors


def best_fit_transform_weight(A, B, W_A, W_B):
    # assert A.shape == B.shape

    # get number of dimensions
    m = A.shape[1]

    # translate points to their centroids
    # centroid_A = np.mean(A, axis=0)
    # centroid_B = np.mean(B, axis=0)
    centroid_A = np.average(A, axis=0, weights=W_A)
    centroid_B = np.average(B, axis=0, weights=W_B)
    AA = A - centroid_A
    BB = B - centroid_B

    # print("A=", A.shape)
    # print("W_A=", W_A.shape)
    # print("B=", B.shape)
    # print("W_B=", W_B.shape) 

    # rotation matrix
    # H = np.dot(AA.T, BB)
    H = np.dot((AA * W_A[:, None]).T, BB) 
    U, S, Vt = np.linalg.svd(H)
    R = np.dot(Vt.T, U.T)

    # special reflection case
    if np.linalg.det(R) < 0:
       Vt[m-1,:] *= -1
       R = np.dot(Vt.T, U.T)

    # translation
    t = centroid_B.T - np.dot(R,centroid_A.T)

    # homogeneous transformation
    T = np.identity(m+1)
    T[:m, :m] = R
    T[:m, m] = t

    return T, R, t


def nearest_neighbor(src, dst):
    # assert src.shape == dst.shape

    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(dst)
    distances, indices = neigh.kneighbors(src, return_distance=True)
    return distances.ravel(), indices.ravel()


def icp_weight(A, B, weight_A, weight_B, init_pose=None, max_iterations=20, tolerance=0.001):
    # assert A.shape == B.shape

    # get number of dimensions
    m = A.shape[1]

    # make points homogeneous, copy them to maintain the originals
    src = np.ones((m+1,A.shape[0]))
    dst = np.ones((m+1,B.shape[0]))
    src[:m,:] = np.copy(A.T)
    dst[:m,:] = np.copy(B.T)

    # print("point_A", A.shape)
    # print("point_B", B.shape)
    # print("weight_A", weight_A.shape)
    # print("weight_B", weight_B.shape)
    # print("src=", src.shape)
    # print("dst=", dst.shape)

    # apply the initial pose estimation
    if init_pose is not None:
        src = np.dot(init_pose, src)

    prev_error = 0

    for i in range(max_iterations):
        # find the nearest neighbors between the current source and destination points
        distances, indices = nearest_neighbor(src[:m,:].T, dst[:m,:].T)

        # print("A=", src[:m,:].T.shape)
        # print("W_A=", weight_A.shape)
        # print("B=", dst[:m,indices].T.shape)
        # print("W_B=", weight_B[indices].shape) 

        # compute the transformation between the current source and nearest destination points
        T,_,_ = best_fit_transform_weight(src[:m,:].T, dst[:m,indices].T, weight_A, weight_B[indices])
        
        # update the current source
        src = np.dot(T, src)

        # check error
        mean_error = np.mean(distances)
        if np.abs(prev_error - mean_error) < tolerance:
            break
        prev_error = mean_error

    # calculate final transformation
    T,_,_ = best_fit_transform_weight(A, src[:m,:].T, weight_A, weight_A)

    return T, distances, i

# 两种ICP方法的实现

In [83]:
point_A, point_B, label_A, label_B = readpoint('01', 100)
T, distances, i = icp(point_A, point_B)
T_weight, distances_weight, i_weight = icp_weight(point_A, point_B, label_A, label_B)

distances_ori, indices_ori = nearest_neighbor(point_A, point_B)
print('ori_dis=', np.sum(distances_ori))
# print('T', T)
print('i=', i)
print('total_distances=', np.sum(distances))
print('i_weight=', i_weight)
print('total_distances_weight=', np.sum(distances_weight))


ori_dis= 26090.346287868728
i= 3
total_distances= 26576.908076246116
i_weight= 5
total_distances_weight= 29698.010800424043


# 可视化

In [84]:
import copy
import numpy as np
# o3d可视化点云配准结果
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
    
point_cloud_A = o3d.geometry.PointCloud()
point_cloud_B = o3d.geometry.PointCloud()
point_cloud_A.points = o3d.utility.Vector3dVector(point_A)
point_cloud_B.points = o3d.utility.Vector3dVector(point_B)

draw_registration_result(point_cloud_A, point_cloud_B, T)
draw_registration_result(point_cloud_A, point_cloud_B, T_weight)

path_gt = '../datasets/dataset/sequences/01/trans_data/000101.bin'
data_gt = np.fromfile(path_gt, dtype=np.float32)
data_gt = data_gt.reshape((-1, 3))
point_cloud_gt = o3d.geometry.PointCloud()
point_cloud_gt.points = o3d.utility.Vector3dVector(data_gt)
point_cloud_gt.paint_uniform_color([1, 0.706, 0])
point_cloud_A.paint_uniform_color([0, 0.651, 0.929])
o3d.visualization.draw_geometries([point_cloud_gt, point_cloud_A])